# After the Deployment

## Intro
Once you deploy your CrewAI project using the **CrewAI Enterprise platform**, it automatically gives you **API endpoints**.

Think of an **API endpoint** as a doorway that other apps—like a website, chatbot, or mobile app—can use to talk to your CrewAI project.

In the screenshot, you can see two important endpoints:

* `/inputs`: This lets you ask, “What information does this Crew need to run?”
* `/kickoff`: This starts the Crew and gets it working with the input you send.

So once your Crew is online, you can connect it to other tools or interfaces by using these API links—no extra setup required. It makes it easy to use your AI agents in real-world applications.

It works **very similarly to FastAPI** when it comes to interacting with APIs. Once your Crew is deployed, it exposes **API endpoints**, and you can connect to them just like you would with any FastAPI app.

## Frontend Integration Tips
* You can use these API endpoints with any frontend framework (React, Next.js, Vue, etc.).
* Remember to secure your **Bearer Token** (a Bearer Token is typically used for authentication and authorization when your app communicates with external services or APIs) using a backend proxy (don’t expose it directly in production).
* For a very simple example using dynamic UIs, you can call `/inputs` to build a form and then send data to `/kickoff`.

## Demo Example: connecting the API endpoints with a React frontend
This is not a real example, just a quick demo to show you the way. Here's a **step-by-step guide** on how to connect your **CrewAI API endpoints** with a **React frontend** to:

1. Fetch required inputs from `/inputs`
2. Submit the user’s data to `/kickoff`

#### Step 1: Setup Your React Project (if not already)

```bash
npx create-react-app crewai-frontend
cd crewai-frontend
npm start
```


#### Step 2: Create a Component to Interact with the API

Create a file like `CrewAIForm.js`:

```jsx
import React, { useEffect, useState } from "react";

const API_URL = "https://latest-ai-development-7860af..."; // Your deployment URL
const BEARER_TOKEN = "4bd20c57ab4"; // Replace with your actual token

function CrewAIForm() {
  const [inputFields, setInputFields] = useState([]);
  const [formData, setFormData] = useState({});
  const [result, setResult] = useState(null);

  // Fetch input fields from /inputs
  useEffect(() => {
    fetch(`${API_URL}/inputs`, {
      method: "POST",
      headers: {
        "Authorization": `Bearer ${BEARER_TOKEN}`,
        "Content-Type": "application/json"
      }
    })
      .then(res => res.json())
      .then(data => {
        setInputFields(data);
        const initialForm = {};
        data.forEach(field => {
          initialForm[field.name] = "";
        });
        setFormData(initialForm);
      });
  }, []);

  // Handle form changes
  const handleChange = (e) => {
    setFormData({ ...formData, [e.target.name]: e.target.value });
  };

  // Handle form submission to /kickoff
  const handleSubmit = async (e) => {
    e.preventDefault();
    const response = await fetch(`${API_URL}/kickoff`, {
      method: "POST",
      headers: {
        "Authorization": `Bearer ${BEARER_TOKEN}`,
        "Content-Type": "application/json"
      },
      body: JSON.stringify(formData)
    });

    const result = await response.json();
    setResult(result);
  };

  return (
    <div>
      <h2>Run Your CrewAI</h2>
      <form onSubmit={handleSubmit}>
        {inputFields.map(field => (
          <div key={field.name}>
            <label>{field.name}</label>
            <input
              type="text"
              name={field.name}
              value={formData[field.name]}
              onChange={handleChange}
              required
            />
          </div>
        ))}
        <button type="submit">Run Crew</button>
      </form>

      {result && (
        <div>
          <h3>Result:</h3>
          <pre>{JSON.stringify(result, null, 2)}</pre>
        </div>
      )}
    </div>
  );
}

export default CrewAIForm;
```


#### Step 3: Import It in `App.js`

```jsx
import React from 'react';
import CrewAIForm from './CrewAIForm';

function App() {
  return (
    <div className="App">
      <CrewAIForm />
    </div>
  );
}

export default App;
```

#### Result

* When the page loads, it fetches the required fields from `/inputs`.
* It displays a dynamic form.
* On submit, it sends your input to `/kickoff`.
* The response from your CrewAI flow is shown in the UI.